In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///db/college.sqlite', echo=True)
conn = engine.connect()
meta = db.MetaData()

In [3]:
from sqlalchemy import Table, Column, Integer, String

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)

ins = students.insert()
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
conn = engine.connect()
result = conn.execute(ins)

2023-06-19 05:04:45,699 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 05:04:45,699 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2023-06-19 05:04:45,700 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ('Ravi', 'Kapoor')


In [4]:
conn.execute(students.insert(), [
   {'name':'Rajiv', 'lastname' : 'Khanna'},
   {'name':'Komal','lastname' : 'Bhandari'},
   {'name':'Abdul','lastname' : 'Sattar'},
   {'name':'Priya','lastname' : 'Rajhans'},
])

2023-06-19 05:04:51,521 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2023-06-19 05:04:51,522 INFO sqlalchemy.engine.Engine [generated in 0.00082s] [('Rajiv', 'Khanna'), ('Komal', 'Bhandari'), ('Abdul', 'Sattar'), ('Priya', 'Rajhans')]


In [5]:
s = students.select()
result = conn.execute(s)

for row in result:
   print(row)

2023-06-19 05:04:53,373 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2023-06-19 05:04:53,374 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ()
(1, 'Ravi', 'Kapoor')
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')
(4, 'Abdul', 'Sattar')
(5, 'Priya', 'Rajhans')


In [6]:
s = students.select().where(students.c.id>2)
result = conn.execute(s)

for row in result:
   print (row)

2023-06-19 05:05:16,174 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id > ?
2023-06-19 05:05:16,175 INFO sqlalchemy.engine.Engine [generated in 0.00091s] (2,)
(3, 'Komal', 'Bhandari')
(4, 'Abdul', 'Sattar')
(5, 'Priya', 'Rajhans')


In [15]:
s = students.select()
result = conn.execute(s)
row = result.fetchone()
for row in result:
   print(row)

2023-06-19 05:07:08,027 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2023-06-19 05:07:08,028 INFO sqlalchemy.engine.Engine [cached since 134.7s ago] ()
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')
(4, 'Abdul', 'Sattar')
(5, 'Priya', 'Rajhans')


In [18]:
# from sqlalchemy.sql import select
# s = select([students])
# result = conn.execute(s)

In [22]:
from sqlalchemy import text
t = text("SELECT * FROM students")
result = conn.execute(t)

2023-06-19 05:08:47,308 INFO sqlalchemy.engine.Engine SELECT * FROM students
2023-06-19 05:08:47,308 INFO sqlalchemy.engine.Engine [cached since 46.03s ago] ()


In [23]:
from sqlalchemy.sql import text
s = text("select students.name, students.lastname from students where students.name between :x and :y")
conn.execute(s, x = 'A', y = 'L').fetchall()

TypeError: execute() got an unexpected keyword argument 'x'

In [26]:
from sqlalchemy import bindparam
stmt = text("SELECT * FROM students WHERE students.name BETWEEN :x AND :y")

stmt = stmt.bindparams(
   bindparam("x", type_= String), 
   bindparam("y", type_= String)
)

result = conn.execute(stmt, {"x": "A", "y": "L"})


2023-06-19 05:10:06,061 INFO sqlalchemy.engine.Engine SELECT * FROM students WHERE students.name BETWEEN ? AND ?
2023-06-19 05:10:06,062 INFO sqlalchemy.engine.Engine [cached since 5.975s ago] ('A', 'L')


In [27]:
# The text() function also be produces fragments of SQL within a select() object that 
# accepts text() objects as an arguments. The “geometry” of the statement is provided by 
# select() construct , and the textual content by text() construct. We can build a statement 
# without the need to refer to any pre-established Table metadata. 

from sqlalchemy.sql import select
s = select([text("students.name, students.lastname from students")]).where(text("students.name between :x and :y"))
conn.execute(s, x = 'A', y = 'L').fetchall()

ArgumentError: Column expression, FROM clause, or other columns clause element expected, got [<sqlalchemy.sql.elements.TextClause object at 0x7fd84b0891c0>]. Did you mean to say select(<sqlalchemy.sql.elements.TextClause object at 0x7fd84b0891c0>)?